In [1]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from time import sleep, time
from random import randint
from warnings import warn
from IPython.core.display import clear_output
from selenium import webdriver

In [4]:
variables = ['commits','branches', 'release', 'contributors','watchers', 'forks','stars', 'issues_open', 'issues_closed']

In [2]:
df_repos = pd.read_csv('Lista de repositorios.csv')
full_names = df_repos['Nome_do_repositorio']

In [ ]:
full_names = df_repos['Nome_do_repositorio']

full_names_test = full_names[0:4]

requests = 0
start_time = time()
elapsed_time = 0

for name_repo in full_names_test:
    url_git = 'https://www.github.com/'.format(name_repo)
    response = get(url_git)

    sleep(randint(1,6))

    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    if requests > len(full_names):
        warn('Numero de requests maior que o espperado')  
        break
    
    pagina_soup = BeautifulSoup(response.text, 'html.parser')
    nomes_servidores = pagina_soup.select("div #listagem table tr td a")
    

In [33]:
full_names_test = full_names[0:3]

url_git = 'http://www.github.com/{}'.format(full_names_test[1])
browser = webdriver.PhantomJS(service_args=['--ignore-ssl-errors=true'])
browser.get(url_git)

html = browser.page_source
pagina_soup = BeautifulSoup(html, 'lxml')

#pagina_soup = BeautifulSoup(response.text, 'html.parser')
#commits = pagina_soup.select("li.commits span.num")[0].text
#branches = pagina_soup.select("a[href='/"+full_names_test[0]+"/branches'] span.num")[0].text
#realeases = pagina_soup.select("a[href='/"+full_names_test[0]+"/releases'] span.num")[0].text
#contributors = pagina_soup.select("a[href='/"+full_names_test[1]+"/graphs/contributors']")[0].text
#watchers = pagina_soup.select("a[href='/"+full_names_test[0]+"/watchers']")[0].text
#forks = pagina_soup.select("a[href='/"+full_names_test[0]+"/network']")[0].text
#stars = pagina_soup.select("a[href='/"+full_names_test[0]+"/stargazers']")[0].text

#url_git_issues = 'http://www.github.com/{}/issues'.format(full_names_test[0])
#browser.get(url_git_issues)

#html = browser.page_source
#pagina_soup = BeautifulSoup(html, 'lxml')
                            
#issues_open = pagina_soup.select("div.issues-listing div.table-list-filters a")[0].text
#issues_closed = pagina_soup.select("div.issues-listing div.table-list-filters a")[1].text

print((contributors))




      66
    
    contributors



In [87]:
"a[href='/"+full_names_test[0]+"/stargazers']"
("a[href='/"+full_names_test[0]+"/issues?q=is%3Aopen+is%3Aissue']")

"a[href='/mojombo/grit/issues?q=is%3Aopen+is%3Aissue']"

In [65]:
full_names_test

0         mojombo/grit
1     wycats/merb-core
2    rubinius/rubinius
Name: Nome_do_repositorio, dtype: object